<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lab-2:-Matrix-factorization" data-toc-modified-id="Lab-2:-Matrix-factorization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Lab 2: Matrix factorization</strong></a></span></li><li><span><a href="#Abstract" data-toc-modified-id="Abstract-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong>Abstract</strong></a></span></li><li><span><a href="#Set-up-environment" data-toc-modified-id="Set-up-environment-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>Set up environment</strong></a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><strong>Introduction</strong></a></span><ul class="toc-item"><li><span><a href="#Sparse-matrix-vector-product" data-toc-modified-id="Sparse-matrix-vector-product-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Sparse matrix-vector product</a></span></li><li><span><a href="#QR-factorization" data-toc-modified-id="QR-factorization-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>QR factorization</a></span></li><li><span><a href="#Direct-solver-Ax=b" data-toc-modified-id="Direct-solver-Ax=b-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Direct solver Ax=b</a></span></li><li><span><a href="#QR-eigenvalue-algorithm" data-toc-modified-id="QR-eigenvalue-algorithm-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>QR eigenvalue algorithm</a></span></li></ul></li><li><span><a href="#Methods" data-toc-modified-id="Methods-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><strong>Methods</strong></a></span><ul class="toc-item"><li><span><a href="#Sparse-matrix-vector-product" data-toc-modified-id="Sparse-matrix-vector-product-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Sparse matrix-vector product</a></span></li><li><span><a href="#QR-factorization" data-toc-modified-id="QR-factorization-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>QR factorization</a></span></li><li><span><a href="#Direct-solver-Ax=b" data-toc-modified-id="Direct-solver-Ax=b-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Direct solver Ax=b</a></span></li><li><span><a href="#QR-eigenvalue-algorithm" data-toc-modified-id="QR-eigenvalue-algorithm-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>QR eigenvalue algorithm</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><strong>Results</strong></a></span><ul class="toc-item"><li><span><a href="#Sparse-matrix-vector-product" data-toc-modified-id="Sparse-matrix-vector-product-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Sparse matrix-vector product</a></span></li><li><span><a href="#QR-factorization" data-toc-modified-id="QR-factorization-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>QR factorization</a></span></li><li><span><a href="#Direct-solver-Ax=b" data-toc-modified-id="Direct-solver-Ax=b-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Direct solver Ax=b</a></span></li><li><span><a href="#QR-eigenvalue-algorithm" data-toc-modified-id="QR-eigenvalue-algorithm-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>QR eigenvalue algorithm</a></span></li></ul></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-7"><span class="toc-item-num">7&nbsp;&nbsp;</span><strong>Discussion</strong></a></span></li></ul></div>

<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/Volkaa/Lab-2/Volkaa_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Matthis Manthe**

# **Abstract**

In this report, different fundamental algorithm in the domain of linear spaces, like QR factorization and other algorithms based on it, are implemented and tested against the corresponding Numpy functions.

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [31]:
"""This program uses a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [33]:
# Load neccessary modules.
# from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

We implement in this second multiple functions, more advanced matrix-linked functions than in the first lab. Two main themes emerge : sparse matrix algorithms, and manipulations of matrices. All the following definitions are taken from the third part of the course : [DD2363 course content.](https://kth.instructure.com/courses/17068/files/2485155?module_item_id=189862)

## Sparse matrix-vector product

We define a sparse matrix as a matrix $A\in \mathbb{R}^{n*m}$ for which $O(\sqrt{nm})$ values are zeros. If we use the matrix-vector product as for dense matrix, most of the operations done during the computation will be multiplications and sums of zeros, which is not interesting. By using the CRS (Compressed sparse row) format to represent a matrix, we implement an optimised version of the matrix-vector product for sparse matrices.

## QR factorization

As proved in the course content, every non singular matrix $A\in \mathbb{R}^{n*n}$ can be expressed as the product of two matrices $Q,R\in \mathbb{R}^{n*n}$ such that Q is orthogonal and R is upper triangular. The process of finding Q and R is called QR factorization, and we show an implementation of such algorithm, using the Gram-Schmidt method.

## Direct solver Ax=b

Any system of linear equations can be written in matrix form : $Ax=b$, with $A\in \mathbb{R}^{m*n}, x,b\in \mathbb{R}^n$, the goal being to find x, given A and b. This is one of the most known, simple, and important type of equation. We chose to solve it using the previously implemented QR factorization and a simple algorithm to invert an upper triangular matrix called backward substitution.

## QR eigenvalue algorithm

For any complex matrix $A\in \mathbb{C}^{m*n}$, we define its eigenvalues $\lambda_i\in \mathbb{C}$ and associated eigenvectors $x_i\in \mathbb{C}^n$ as the couple that solves the equation $Ax_i=\lambda_ix_i$. We show an implementation of the QR algorithm, mainly based on the previously presented QR factorization which enables to approximate all the eigenvalues and associated eigenvectors of a matrix A. We constrain ourselves to real symmetric matrix, so the eigenvalues and eigenvectors are also real.

# **Methods**

## Sparse matrix-vector product

As stated in the introduction, we use a representation of a matrix A in CRS format. In this format, a matrix is represented as three lists : a first one (val) being the list of the non zero values of the matri, reading the matrix from left to right, then from top to bottom. A second list (col_idx) is the number of the column of the associated non zero value at the same index in the val list. The third list (row_ptr), of size the number of rows of the matrix, is the index in the lists val and col_idx of the first non zero value in the associated row. If there are none, its value is the index of first non zero value of the first next row containing a non zero value. With such format, the matrix-vector product $c=Ax$ with $A\in \mathbb{R}^{m*n}$ and $x\in \mathbb{R}^n$ is $c_i=\sum_{j=\mbox{row_ptr}_i}^{\mbox{row_ptr}_{i+1}}val_j*x_{\mbox{col_idx}_j}$

In this implementation, A must be in the CRS form : a dictionary of the form {'val':val, 'col_idx':col_idx, 'row_ptr':row_ptr} as described above. x is a numpy array and m and n are the sizes of A.

In [5]:
def sparse_matrix_vector_product(A,x,m,n):
    if (n!=len(x)): return np.nan
    result = []
    for i in range(m):
        s = 0
        for j in range(A['row_ptr'][i], A['row_ptr'][i+1]):
            s += A['val'][j-1]*x[A['col_idx'][j-1]-1]
        result.append(s)
    return np.array(result)

## QR factorization

We use the classic Gram-Schmidt orthonormalisation process of creating an orthonormal basis of the range of a matrix A. We can quickly define this process as an iterative algorithm, each iteration j creating a new vector $q_j$ of the orthonormal basis as follow : $v_j=a_{:j}-\sum_{i=1}^{j-1}(a_{:j},q_i)q_i$, and we have $q_j=\frac{v_j}{||v_j||}$, we use here the 2-norm for vectors. In this new orthonormal basis, we can write each column vector of A : $a_{:i}=\sum_{j=1}^{i}r_{ji}q_i$, with $r_{ji}=(a_{:j},q_i)$ if $i\neq j$ and $r_{jj}=||v_j||$. We finally have that $A=\begin{bmatrix}
    \vert & ... & \vert \\
    q_1 & ... & q_n \\
    \vert & ... & \vert
\end{bmatrix}*\begin{bmatrix}
r_{11} & r_{12} & ... & r_{1n}\\
0 & r_{22} & ... & r_{2n}\\
...\\
0 & 0 & ... & r_{nn}
\end{bmatrix} = QR$, Q being orthogonal and R upper triangular.

In this implementation, A must be non singular matrix as a numpy array.

In [6]:
def QR_factorization(A):
    Q = np.zeros(A.shape)
    R = np.zeros(A.shape)
    for j in range(A.shape[0]):
        v = A[:,j].copy()
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            v = v - R[i,j]*Q[:,i].copy()
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = v.copy()/R[j,j]
    return Q,R

## Direct solver Ax=b

As stated in the course content, there is an easy way of solving this kind of equation when $A\in \mathbb{R}^{n*n}$ is special. If A is orthogonal, then $A^{-1}=A^T$, and $x=A^T*b$. We use backward substitution when A is upper triangular. Indeed, when $A=\begin{bmatrix}
a_{11} & a_{12} & ... & a_{1n}\\
0 & a_{22} & ... & a_{2n}\\
...\\
0 & 0 & ... & a_{nn}
\end{bmatrix}$, the system of equation is actually $b_i=\sum_{j=i}^na_{ij}x_j$, so it's clear that $x_n=b_n/a_{nn}$, and more generally that $x_i=(b_i-\sum_{i+1}^{n}a_{ij}x_j)/a_{ii}$. Thus, we just compute straight forward the solution x in this case. We previously implemented the QR factorization, which means that we can reduce any equation from $Ax=b$ to $QRx=b$, leading to $Rx=Q^Tb$, and by backward substitution $x=R^{-1}Q^Tb$.

In this implementation, A must be a non singular matrix as a numpy array, and b a vector as a numpy array of size the row size of A.

In [7]:
def backward_substitution(U,b):
    n = len(b)
    x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1][n-1]
    for i in [n-2-j for j in range(n-1)]:
        s = 0
        for j in range(i+1,n):
            s += U[i,j]*x[j]
        x[i] = (b[i] - s)/U[i,i]
    return x

def direct_solver(A,b):
    Q,R = QR_factorization(A)
    b = np.matmul(np.transpose(Q),b)
    return backward_substitution(R,b)

## QR eigenvalue algorithm

To approximate the eigenvalues and associated eigenvectors of a matrix $A\in \mathbb{R}^{n*n}$, we use the QR algorithm, based on the QR factorization implemented above. By expressing $A=QR$ with Q orthogonal and R upper triangular, we have that $RQ=Q^{-1}AQ$ which is similar to A, and so they have the same eigenvalues and eigenvectors. If we write $A^{(k)}=R^{(k)}Q^{(k)}$ with $A^{(k-1)}=Q^{(k)}R^{(k)}$, and starting with $A^{(0)}=A$, at each iteration k we have a new matrix $A^{(k)}=R^{(k)}Q^{(k)}=(Q^{(k)})^{-1}A^{(k-1)}Q^{(k)}$ similar to $A^{(k-1)}$, and so similar to A. As stated in course content, we can prove that this iterative algorithm converges towards $A^{(k)}$ being diagonal with the eigenvalues on the diagonal when A is normal, and $U^{(k)}=\prod_{i=1}^kQ^{(i)}$ being the matrix of the associated eigenvectors.

In this implementation, A must be a square matrix as numpy array, and nb_iteration an integer, defining the number of iteration to compute. This returns $A^{(k)}$ with the eigenvalues on the diagonal, and $U^{(k)}$ the matrix of eigenvectors on each column.

In [8]:
def QR_algorithm(A,nb_iteration):
    U = np.identity(A.shape[0])
    for k in range(nb_iteration):
        Q,R = QR_factorization(A)
        A = np.matmul(R,Q)
        U = np.matmul(U.copy(),Q)
    return A,U

# **Results**

## Sparse matrix-vector product

To test the correctness of the algorithm, we implement a function to generate random sparse matrix, here with the number of non-zero values being between $\frac{\sqrt(m*n)}{2}$ and $(\sqrt(m*n))*2$. Two functions to translate a matrix from the CRS format to a numpy array, and vis versa are also implemented, making the generation of random matrix easier. We compare our results to those of the numpy matrix-vector product.

In [9]:
def build_np_matrix_from_CRS(A,m,n):
    result = np.zeros((m,n))
    current_row = 0
    for i in range(len(A['val'])):
        while A['row_ptr'][current_row+1] == i+1: current_row+=1
        result[current_row][A['col_idx'][i]-1] = A['val'][i]
    return result

def build_CRS_from_np_matrix(A):
    val = []
    col_idx = []
    row_ptr = np.array(np.zeros(A.shape[0]+1), dtype=int)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i][j]!= 0:
                val.append(A[i][j])
                col_idx.append(j+1)
                if row_ptr[i]==0:row_ptr[i]=len(val)
    row_ptr[-1] = len(val)+1
    for j in [A.shape[0]-i for i in range(A.shape[0]+1)]:
        if row_ptr[j]==0:row_ptr[j]=row_ptr[j+1]
    return {'val':np.array(val), 'col_idx':np.array(col_idx), 'row_ptr':row_ptr}

In [10]:
val = np.array([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3])
col_idx = np.array([1,  2,  4,  2,  3,  3,  3,  4,  5,  5,  6])
row_ptr = np.array([1,  4,  6,  7,  9,  10,  12])
A = {'val':val, 'col_idx':col_idx, 'row_ptr':row_ptr}
m = 6
n = 6
x = np.array([1,2,3,4,5,6])
dense_A = build_np_matrix_from_CRS(A,6,6)
print(dense_A)
print(sparse_matrix_vector_product(A,x,m,n))
print(np.matmul(dense_A,x))
print(sparse_matrix_vector_product(A,x,m,n)==np.matmul(dense_A,x))
print(build_CRS_from_np_matrix(dense_A))

[[3. 2. 0. 2. 0. 0.]
 [0. 2. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 3. 2. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 2. 3.]]
[15  7  3 17  5 28]
[15.  7.  3. 17.  5. 28.]
[ True  True  True  True  True  True]
{'val': array([3., 2., 2., 2., 1., 1., 3., 2., 1., 2., 3.]), 'col_idx': array([1, 2, 4, 2, 3, 3, 3, 4, 5, 5, 6]), 'row_ptr': array([ 1,  4,  6,  7,  9, 10, 12])}


In [11]:
def random_sparse_matrix(nb_val,m,n):
    result = np.random.rand(m,n)*10
    for i in range(m*n-nb_val):
        remove_idx = (np.random.randint(m),np.random.randint(n))
        while (result[remove_idx]==0):remove_idx = (np.random.randint(m),np.random.randint(n))
        result[remove_idx]=0
    return result

In [12]:
def test_sparse_matrix_vector_product():
    m = np.random.randint(2,10)
    n = np.random.randint(2,10)
    A = random_sparse_matrix(
        np.random.randint(
            int(np.sqrt(m*n)/2),
            int(np.sqrt(m*n)*2)),m,n)
    CRS_A = build_CRS_from_np_matrix(A)
    x = np.random.rand(n)*10
    print("Matrix : ", A)
    print("\nCRS form of A : ", CRS_A)
    print("\nVector x : ", x)
    result = sparse_matrix_vector_product(CRS_A,x,m,n)
    print("\nAx = ", result)
    print("\nNumpy matmul result : ", np.matmul(A,x))
    print("\nResult : ", result==np.matmul(A,x))

In [16]:
test_sparse_matrix_vector_product()

Matrix :  [[0.52180942 0.         0.         0.         0.         0.
  0.        ]
 [1.4792793  0.         0.         0.         0.         0.
  0.        ]
 [1.26416253 0.         0.         0.         0.         9.00290413
  0.        ]
 [0.         7.17764806 0.         9.83734066 0.         0.
  0.        ]
 [0.         7.8309858  0.         0.         0.         0.
  0.        ]
 [3.023765   8.14049337 9.25603011 0.         0.         0.
  0.        ]]

CRS form of A :  {'val': array([0.52180942, 1.4792793 , 1.26416253, 9.00290413, 7.17764806,
       9.83734066, 7.8309858 , 3.023765  , 8.14049337, 9.25603011]), 'col_idx': array([1, 1, 1, 6, 2, 4, 2, 1, 2, 3]), 'row_ptr': array([ 1,  2,  3,  5,  7,  8, 11])}

Vector x :  [5.60279094 7.11782439 0.98058601 3.5747667  5.55250536 7.37974632
 2.58264655]

Ax =  [ 2.92358911  8.28809265 73.52198691 86.25543624 55.73958174 83.9604591 ]

Numpy matmul result :  [ 2.92358911  8.28809265 73.52198691 86.25543624 55.73958174 83.9604591 ]

Resu

## QR factorization

To test the correctness of the algorithm, we implement a function to generate random non singular square matrices of size n. We validate our results by verifying that Q is orthogonal, R is upper triangular, and that $QR=A$.

In [17]:
def random_non_singular_square_matrix(n):
    A = np.random.rand(n,n)*10
    while (np.linalg.matrix_rank(A)!=n): A = np.random.rand(n,n)*10
    return A

In [18]:
def test_QR_factorization():
    n = np.random.randint(2,10)
    A = random_non_singular_square_matrix(n)
    Q,R = QR_factorization(A)
    print("A : ", A)
    print("\nQ : ", Q)
    print("\nR : ", R)
    print("\nQR : ", np.matmul(Q,R))
    print("\nR is upper triangular : ", np.allclose(R, np.triu(R)))
    print("\n||Q^TQ-I||_F = ", np.linalg.norm(np.matmul(np.transpose(Q),Q)-np.identity(n)))
    print("\n||QR-A||_F = ", np.linalg.norm(np.matmul(Q,R)-A))

In [19]:
test_QR_factorization()

A :  [[1.38368445 9.22499809 4.17397642]
 [1.82096461 5.43419444 6.43064613]
 [1.62909068 1.3449217  7.73495373]]

Q :  [[ 0.49277864  0.78784515  0.36941742]
 [ 0.64850947 -0.04945483 -0.75959837]
 [ 0.58017642 -0.61388455  0.53529532]]

R :  [[ 2.80792294  8.85030042 10.71481911]
 [ 0.          6.17349623 -1.77794796]
 [ 0.          0.          0.79771581]]

QR :  [[1.38368445 9.22499809 4.17397642]
 [1.82096461 5.43419444 6.43064613]
 [1.62909068 1.3449217  7.73495373]]

R is upper triangular :  True

||Q^TQ-I||_F =  5.283471786934008e-15

||QR-A||_F =  4.440892098500626e-16


## Direct solver Ax=b

To test the correctness of our algorithm, we implement a function to generate random equations of the form $Ax=b$ with a known solution. We validate our results by verifying that $Ax=b$, and that the found solution is indeed the correct known solution. All the generated equations have exactly one solution.

In [20]:
def random_equation():
    n = np.random.randint(2,10)
    A = random_non_singular_square_matrix(n)
    x = np.random.rand(n)*10
    b = np.matmul(A,x)
    return A,x,b

In [21]:
def test_direct_solver():
    A,x,b = random_equation()
    print("A : ", A)
    print("\nb : ", b)
    print("\nManufactured solution : ", x)
    sol = direct_solver(A,b)
    print("\nFound solution : ", sol)
    print("\n||Ax-b|| = ", np.linalg.norm(np.matmul(A,x)-b))
    print("\n||x-y|| = ", np.linalg.norm(x-sol))

In [22]:
test_direct_solver()

A :  [[1.46240909 7.12144705 8.43896124 7.96750045 1.12855752 5.81734657
  7.13391472]
 [0.92703803 9.73405689 3.96198616 4.81598809 5.0377156  5.20125936
  6.70245906]
 [0.67531768 4.48202927 7.71707734 0.62158334 5.51423987 9.22654129
  2.93692111]
 [2.65315006 0.9690547  9.71450228 2.98854904 5.79511685 7.42052287
  9.65550945]
 [4.27678045 4.20172678 8.70314612 4.5866628  9.84712848 1.32350989
  5.98050316]
 [3.56184137 1.47177712 2.25921267 2.03090173 0.14480414 1.85784058
  0.85481835]
 [2.92383955 8.74018233 5.78165355 8.15216119 3.68325582 3.17086619
  4.10165275]]

b :  [293.23585528 249.04517817 203.09701246 256.28779065 229.85725374
  90.38505215 259.86605155]

Manufactured solution :  [6.71574906 7.76428037 6.92241062 8.83008315 1.53895761 8.22359237
 6.97707187]

Found solution :  [6.71574906 7.76428037 6.92241062 8.83008315 1.53895761 8.22359237
 6.97707187]

||Ax-b|| =  0.0

||x-y|| =  3.837446067023546e-12


## QR eigenvalue algorithm

To test the correctness of our algorithm, we implement a function to generate symmetric real matrices. We validate our result by verifying that the eigenvalues $\lambda_i$ are indeed eigenvalues, by proving that $det(A-\lambda_iI)=0$, and that the eigenvectors $v_i$ are the right eigenvectors associated with the right eigenvalues, by proving that $Av_i=\lambda_iv_i$. The number of iteration is set to 100. By testing the algorithm, we see that it is enough when the size of the matrix is 5 or more, but when the size goes up, the precision goes down, and more iteration can be required to have a enough precision.

In [23]:
def random_symmetric_real_matrix():
    n = np.random.randint(2,10)
    A = np.random.rand(n,n)*10
    A = np.matmul(A,np.transpose(A))/2.0
    return A

In [24]:
def test_QR_algorithm():
    k = 100
    A = random_symmetric_real_matrix()
    n = A.shape[0]
    V,U = QR_algorithm(A,k)
    print("A : ", A)
    print("\nEigenvalues found : ", np.diag(V))
    print("\nEigenvectors (columns) found : ", U)
    print("\ndet(A-lambda_i I) = ", [np.linalg.det(A-V[i,i]*np.identity(n)) for i in range(n)])
    print("\n||Av_i - lambda_i v_i|| = ", [np.linalg.norm(np.matmul(A,U[:,i]) - V[i,i]*U[:,i]) for i in range(n)])

In [30]:
test_QR_algorithm()

A :  [[74.50376427 62.45720405 17.83027619 50.82345172 59.54207313]
 [62.45720405 95.10921691 25.17904274 56.16524535 57.69920781]
 [17.83027619 25.17904274 12.97152794 17.7571221  21.58050359]
 [50.82345172 56.16524535 17.7571221  39.85322468 45.24246225]
 [59.54207313 57.69920781 21.58050359 45.24246225 66.27543567]]

Eigenvalues found :  [2.45923585e+02 2.59745603e+01 1.26828076e+01 4.13217669e+00
 3.96705891e-05]

Eigenvectors (columns) found :  [[ 0.50926215 -0.47434252 -0.61484542 -0.1435457   0.34206869]
 [ 0.57042524  0.75439811 -0.05920393  0.2519831   0.19620895]
 [ 0.17503151  0.09216726  0.44100271 -0.81581065  0.3175512 ]
 [ 0.40017697  0.01376082 -0.08964195 -0.29670154 -0.86232334]
 [ 0.4738032  -0.44407039  0.64493483  0.40288998  0.00712386]]

det(A-lambda_i I) =  [-7.78526972682565e-05, -1.626835870180853e-08, 9.472637527058524e-10, -1.0894813918582787e-09, -2.6009503546011557e-09]

||Av_i - lambda_i v_i|| =  [5.3644800141730896e-14, 3.743379820355309e-13, 4.353679252

# **Discussion**

In the end, all the implemented algorithm are working, and there not many implications of that. We could have made these functions more robust, for example by taking into account wrong inputs, but it's not the goal of this lab.